# Systematic search on Web of Science and Scopus

## II. Exporting the databases 

As required by Chapter 13, we will create the database corresponding to the following search equations:

On WoS : 

TS = (("environment" OR "enabling condition*") AND ("greenhouse gas reduct$^*$" OR decarboni?ation OR "climate change mitigation"))

On Scopus :

TITLE-ABS-KEY ( ( "environment"  OR  "enabling condition*" )  AND  ( "greenhouse gas reduct$^*$"  OR  decarboni?ation  OR  "climate change mitigation" ) )

### II.A On the websites

For both WoS and Scopus, select the **Advanced search** mode

On **WoS**, click on the **"All databases"** cursor to observe as many papers as possible :

<tr>
    <td> <img src="img/WOS_search.png" width = "500"/> </td>
    <td> <img src="img/Scopus_search.png" width = "500"/> </td>
</tr>

#### Storing records from WoS

WoS does not impose limits on the number of data that can be saved, however, they can only be saved in batchs of 500. When the search equation leads to thousands of results, it may be a little tedious but it will very rarely take an hour or more to download these results. 

**1.** Click on the **Add to Marked List** button

**2.** Add all the records to the Marked List with the second option

**3.** Go to the Marked List

<tr>
    <td> 1. <img src="img/WOS1_marked_list_selec.png" width = "500"/> </td>
    <td> 2. <img src="img/WOS2_marked_list_details.png" width = "500"/> </td>
    <td> 3. <img src="img/WOS3_go_marked_list.png" width = "500"/> </td>
</tr>

**4.** Create folders on your computer to store all the records in a systematic way

<tr>
    <td> <img src="img/WOS_databases_folders.png" width = "400"/> </td>
    <td> <img src="img/WOS_folders.png" width = "250"/> </td>
</tr>

**5.** Before to start downloading data, think about the nomenclature of each batch  :<br>
I find useful to take the nomenclature "savedrecs0.txt", "savedrecs1.txt", "savedrecs2.txt"... inside each database folder <br>
We will see afterwards the advantages of this with python

<tr>
    <td>  <img src="img/WOS_folders_All_databases.png" width = "500"/> </td>
    <td>  <img src="img/WOS_folders_core_collection.png" width = "500"/> </td>
    <td>  <img src="img/WOS_folders_MEDLINE.png" width = "300"/> </td>
</tr>

**6.** Export all records by batch of 500 publications and choose the **Other File Formats** option

**7.** Select the **Tab-delimited Mac, UTF-8** format

**8.** Rename the downloaded file and sort it in the corresponding database folder

**9.** Repet the process for each batch from each database

**Actually, we download first the database which contains all the publications but little information on each (e.g. authors keywords) and all the other databases afterwards because they contain much more information on each of the publications**

<tr>
    <td> 6. <img src="img/WOS4_export_batch1.png" width = "500"/> </td>
    <td> 7. <img src="img/WOS5_export_batch1_details.png" width = "500"/> </td>
</tr>

#### Storing records from Scopus

Contrary to WoS, Scopus imposes limits on the number of data that can be saved. Actually, the recording can not exceed 2000 publication per search equation. 

To download all the results, it is then necessary to divide the search equation into several equations with more constraints so as not to exceed the 2000 results. The equations overlap is not a problem, we will then see how to merge the sub-databases to avoid duplicates. 

Here, we have only 1166 results so we don't have to divide the search equation to record all of these ones. 

**1.** Select **All** results and click on the **Export** button

**2.** Select all the information available and export results in **CSV** format

<tr>
    <td> 1. <img src="img/Scopus_export.png" width = "550"/> </td>
    <td> 2. <img src="img/Scopus_export_details.png" width = "500"/> </td>
</tr>

By symetry with WOS folders, store the downloaded file in a *SCOPUS* folder as *scopus0.csv*

### II.B Merging the sub-databases from WoS

Import Python librairies needed for merging and saving data:

In [1]:
import pandas as pd 
import numpy as np
import joblib

#### Concatenate records from databases folders containing multiple files

Define concatenation function with two arguments :

* *wos_database* the name of the database folder
* *n* the number of files in the database folder

In [2]:
def concat_wos(wos_database_folder,n):
    mainpath= "./Exported/WOS/" + wos_database_folder + "/savedrecs"
    frames =[]
    for k in range(n): 
        file = mainpath + str(k) +".txt"
        df = pd.read_csv(file, sep = '\t', header = 0, index_col=False)
        frames.append(df)  
    Df = pd.concat(frames, sort=False)    
    return(Df)

Concatenate files from "All_databases" : <br> 
(There are only **2** batchs of 500 publications recorded for this search equation, so just make the concatenation for these ones)

In [3]:
all_databases_folder = "All_databases"
df_wos = concat_wos(all_databases_folder, 2)
print("%d X %d dataframe" % (len(df_wos), len(df_wos.columns) ))

1000 X 57 dataframe


Create a column in the file from "All_databases" that will contain authors keywords : 

In [4]:
df_wos["DE"] = np.nan

Concatenate batchs from "Core_collection" :

In [5]:
core_collection_folder = "Core_collection"
df1 = concat_wos(core_collection_folder, 2)
print("%d X %d dataframe" % (len(df1), len(df1.columns) ))

945 X 67 dataframe


#### Read records from databases which contain only one file

In [6]:
df2 = pd.read_csv("./Exported/WOS/Current_Contents_Connect/savedrecs0.txt", sep = '\t', header = 0, index_col=False)
df3 = pd.read_csv("./Exported/WOS/KCI-Korean/savedrecs0.txt", sep = '\t', header = 0, index_col=False)
df4 = pd.read_csv("./Exported/WOS/MEDLINE/savedrecs0.txt", sep = '\t', header = 0, index_col=False)
df5 = pd.read_csv("./Exported/WOS/Russian_Science/savedrecs0.txt", sep = '\t', header = 0, index_col=False)
df6 = pd.read_csv("./Exported/WOS/SciELO/savedrecs0.txt", sep = '\t', header = 0, index_col=False)

#### Merge all information from the different databases to the one that contains all the publications

Take the WoS ID number (*UT*) as index to make the merge : 

In [7]:
df_wos.set_index("UT", inplace = True)
df1.set_index("UT", inplace = True)
df2.set_index("UT", inplace = True)
df3.set_index("UT", inplace = True)
df4.set_index("UT", inplace = True)
df5.set_index("UT", inplace = True)
df6.set_index("UT", inplace = True)

Update the database containing all the papers with the 6 databases on which WoS is based :

In [8]:
df_wos.update(df1)
df_wos.update(df2)
df_wos.update(df3)
df_wos.update(df4)
df_wos.update(df5)
df_wos.update(df6)

In [9]:
df_wos.reset_index(0, inplace =True)

#### Rename columns to have same names for WoS and Scopus databases

(You can find original names of WoS columns [here](https://images.webofknowledge.com/images/help/WOS/hs_wos_fieldtags.html) )

In [10]:
df_wos.rename(columns ={'UT': 'WOS_number'}, inplace=True)
df_wos.rename(columns ={'TI': 'title'}, inplace=True)
df_wos.rename(columns ={'AU': 'authors'}, inplace=True)
df_wos.rename(columns ={'SO': 'source'}, inplace=True)
df_wos.rename(columns ={'DI': 'doi'}, inplace=True)
df_wos.rename(columns ={'PY': 'publication_year'}, inplace=True)
df_wos.rename(columns ={'AB': 'abstract'}, inplace=True)
df_wos.rename(columns ={'DE': 'author_keywords'}, inplace=True)

#### Select only the renamed columns 

In [11]:
col=['WOS_number','title','authors','source','doi','publication_year','abstract','author_keywords']
df_wos = df_wos.reindex(columns=col)
df_wos.head()

,WOS_number,title,authors,source,doi,publication_year,abstract,author_keywords
0,WOS:000491647400001,Cultural Lock-in and Mitigating Greenhouse Gas...,"Burton, RJF; Farstad, M",SOCIOLOGIA RURALIS,10.1111/soru.12277,2019.0,Meeting targets for reducing greenhouse gas em...,NaN
1,WOS:000482491500071,Unpacking the determinants of cross-border pri...,"Ragosa, G; Warren, P",JOURNAL OF CLEANER PRODUCTION,10.1016/j.jclepro.2019.06.166,2019.0,Private finance has emerged as a fundamental c...,Renewable energy; Developing countries; Climat...
2,WOS:000481589100039,Effects of urbanization on winter wind chill c...,"Lin, LJ; Luo, M; Chan, TO; Ge, EJ; Liu, XP; Zh...",SCIENCE OF THE TOTAL ENVIRONMENT,10.1016/j.scitotenv.2019.06.145,2019.0,Human-perceived wind chill describes the combi...,Wind chill; Human perceived temperature; Urban...
3,WOS:000489349300001,A life-cycle assessment model for zero emissio...,"Lausselet, C; Ellingsen, LAW; Stromman, AH; Br...",JOURNAL OF INDUSTRIAL ECOLOGY,10.1111/jiec.12960,2019.0,Buildings represent a critical piece of a low-...,buildings; built environment; functional unit;...
4,WOS:000483406000045,Integrated sustainability assessment for a bio...,"Jin, E; Mendis, GP; Sutherland, JW",JOURNAL OF CLEANER PRODUCTION,10.1016/j.jclepro.2019.06.205,2019.0,Cellulosic biofuels produced from energy crops...,Sustainability; System dynamics; Bioenergy sys...


### II.C Fusion of the sub-databases from Scopus

We don't need a fusion function for Scopus in this case because the search equation led to less than 2000 publications. However, if we had to find more constrained equations, it would have been very usefull. **So let do as if we had to deal with sub-databases from Scopus to learn how to delete duplicates:** 

#### Concatenate the records from each equation

Define concatenation function with one argument:
 * *n* the number of files in the SCOPUS folder

In [12]:
def fusion_scop(n):
    mainpath= "./Exported/SCOPUS/scopus"
    frames =[]
    for k in range(n): 
        file = mainpath + str(k) +".csv"
        df = pd.read_csv(file)
        frames.append(df)  
    Df = pd.concat(frames, sort=False)    
    return(Df)

Concatenate the files :

In [13]:
df_scop = fusion_scop(1)
print("%d X %d dataframe" % (len(df_scop), len(df_scop.columns) ))

1166 X 53 dataframe


#### Use the Scopus ID as a key to delete duplicates (if there are different sub-databases, which is not the case here)

In [14]:
df_scopus = df_scop.groupby(['EID']).last().reset_index()
print("%d X %d dataframe" % (len(df_scop), len(df_scop.columns) ))

1166 X 53 dataframe


#### Rename columns to have same names for WoS and Scopus databases:

In [15]:
df_scopus.rename(columns ={'EID': 'scopus_number'}, inplace=True)
df_scopus.rename(columns ={'Title': 'title'}, inplace=True)
df_scopus.rename(columns ={'Authors': 'authors'}, inplace=True)
df_scopus.rename(columns ={'Source title': 'source'}, inplace=True)
df_scopus.rename(columns ={'DOI': 'doi'}, inplace=True)
df_scopus.rename(columns ={'Year': 'publication_year'}, inplace=True)
df_scopus.rename(columns ={'Abstract': 'abstract'}, inplace=True)
df_scopus.rename(columns ={'Author Keywords': 'author_keywords'}, inplace=True)

#### Select only the renamed columns :

In [16]:
col=['scopus_number','title','authors','source','doi','publication_year','abstract','author_keywords']
df_scopus = df_scopus.reindex(columns=col)
df_scopus.head()

,scopus_number,title,authors,source,doi,publication_year,abstract,author_keywords
0,2-s2.0-0012952540,Technology evolution and energy modelling: Ove...,"Capros P., Vouyoukas E.L.",International Journal of Global Energy Issues,NaN,2000,This paper provides an overview of the researc...,Climate change; Endogenous technology progress...
1,2-s2.0-0019698977,MECHANICAL PROPERTY CHANGES OF HASTELLOY X EXP...,McCoy Jr. H.E.,NaN,NaN,1981,[No abstract available],NaN
2,2-s2.0-0024755686,Structural deformation of black shale beds res...,Yusupova I.F.,Doklady. Earth science sections,NaN,1989,Any significant decrease in volume or thicknes...,NaN
3,2-s2.0-0025839071,Global climate change assessment in Washington...,Canning Douglas J.,Coastal Zone: Proceedings of the Symposium on ...,NaN,1991,Two global climate change assessment projects ...,NaN
4,2-s2.0-0026173249,New reports on energy and the environment,"Nadel Steven, Shepherd Michael, Greenberg Stev...",Strategic Planning for Energy and the Environment,NaN,1991,This note reviews new reports on energy and en...,NaN


### Store databases (df_wos and df_scopus) in pkl format

In [17]:
joblib.dump((df_wos,df_scopus), "df_wos_df_scopus.pkl")

['df_wos_df_scopus.pkl']